In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Grid initialization
DR = 1e-5  # 10^-3cm spatial grid size
R_SIZE = 5e4  # 50cm radius
DT = 1e-3
T_SIZE = 2e4  # 20s max iteration time
e = 1.902e-19  # charge of electron
M_E = 9.11e-31  # electron mass
R_S = 0.285  # beam radius
L_S=0.005  # parameter that appeared in Sb calculation

In [ ]:
def generate_profile(Te_profile, n0_profile, current_profile, density_init, B, diffusion_mode, diffusion_coef_ratio=-0.5, n0_vdist=None):
    
    N_prev = density_init
    # define variables and coefficients
    Omega = e * B / M_E  # Gyrofrequency

 
    # ---------- auxiliary inner functions ----------
    def D_F(j):
        # WARNING: in classical mode, density is evaluated with old density (i.e. explicit method)
        if (diffusion_mode == 'classical'):
            # Columb logarithm is 22, density expressed converted to m-3 unit
            nu_ei = 2.9e-6 * 22 * N_prev(j) * 1e-6 / Te_profile[j]**1.5
            return nu_ei * Te_profile[j] / (M_E * Omega**2)
        elif (diffusion_mode == 'Bohm'):
            # nu_f/Omega = 1 / 16 for Bohm diffusion
            return 16 * Te_profile[j] / (M_E * Omega)
        else:
            raise ValueError('unidentified diffusion mode')
    
    def D_T(j):
        return D_F(j) * diffusion_coef_ratio
    
    def D1(j):
        return j * DR * D_F(j)

    def D2(j):
        if (j < np.size(Te_profile) - 1 and j > 0):
            T1 = Te_profile[j+1]
            T0 = Te_profile[j-1]
            return j * DR * D_T(j) * (np.log(T1)-np.log(T0) / (2 * DR))
        elif (j == np.size(Te_profile) - 1):
            T1 = Te_profile[j]
            T0 = Te_profile[j-1]
            return j * DR * D_T(j) * (np.log(T1)-np.log(T0) / DR)
        elif (j == 0):
            T1 = Te_profile[j+1]
            T0 = Te_profile[j]
            return j * DR * D_T(j) * (np.log(T1)-np.log(T0) / DR)
        else:
            raise ValueError('coefficient evaluation outside profile range')
        
    def Sb(r, S):
        '''Calculate radial dependence of beam ionization term using formula:
        Sb(r)=sb(0){1-tanh[(r-r_s)/L_s]}/2. S is the total rate of ionization.'''
        # Analytical solution of the integral of the above equation from 0 to r_s, 
        # with integral equals S
        S0 = S / (np.pi * (R_S**2/2 - np.log(np.cosh(R_S/L_S))))
        return S0 * (1 - np.tanh((r-R_S)/L_S)) / 2
    
    def cross_section(E):
        '''Electron ionization cross section (sigma) of He. Assume ground state
         Helium. See Janev et.al., 'Elementary Processes in 
         Hydrogen-Helium Plasmas' (1987). p.90,247.
        '''
        coef = np.array([-1.86452e3, 2.20004e3, -1.13502e3,
                         3.27537e2, -5.78277e1, 6.39886,
                         -4.33709e-1, 1.64789e-2, -2.69004e-4])
        sigma = coef[0]
        for k in range(1,9):
            sigma += coef[k] * E ** k
        return sigma
    
    def R_p(T):
        '''Ionization rate coefficient (<sigma*v>, R_p) of He. 
        Assume ground state Helium. T in eV. See Janev et.al., 
        'Elementary Processes in Hydrogen-Helium Plasmas' (1987). p.90,263.
        '''
        coef = np.array([-4.40986e1, 2.39160e1, -1.07532e1,
                         3.05804e0, -5.68512e-1, 6.79539e-2,
                         -5.00906e-3, 2.06724e-4, -3.64916e-6])
        Rp = coef[0]
        for k in range(1, 9):
            Rp += coef[k] * (np.log(T)) ** k
        return Rp

    # ------------ start of main code --------------

    E = np.empty(R_SIZE)
    F = np.empty(R_SIZE)
    N = np.empty(R_SIZE)
    for k in range(1, T_SIZE):

        # impose Neumann boundary conditions
        N[0] = 0
        N[1] = 0  # zero gradient at boundary

        for j in range(1, R_SIZE):
            # calculate local coefficients
            ita_L = 1.25e-3  # reported end loss rate 1.25ms
            Rp = R_p(Te_profile[j])  # assume Maxwellian distribution of electrons

            # calculate tridiagonal coefficients
            a = -1 / (2*j*DR**2) * ((D1(j)+D1(j-1)) / DR - (D2(j-1)+D2(j)) / 2)
            b = 1 / (2*j*DR**2) * ((D1(j+1)+2*D1(j)+D1(j-1)) / DR - (D2(j+1)-D2(j-1)) / 2) + 1 / ita_L + Rp * n0_profile[j] + 1 / DT
            c = -1 / (2*j*(DR)**2) * ((D1(j+1)+D1(j)) / DR + (D2(j+1)+D2(j)) / 2)
            d = Sb(j, current_profile[k]) + N_prev[j] / DT
            N[j+1] = (d + b * N[j] - a * N[j-1]) / c

        N_prev = N

       

    return N
